In [ ]:
pip install numpy matplotlib opencv-python svgwrite cairosvg shapely

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import svgwrite
import os

def read_csv(csv_path):
    """Read CSV and parse polyline data."""
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',', skip_header=1)
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def create_blank_image(img_shape):
    """Create a blank white image of the given shape."""
    return np.ones(img_shape, dtype=np.uint8) * 255

def composite_shapes(paths_XYs, img_shape, shape_order):
    """Composite all shapes into a single image."""
    blank_image = create_blank_image(img_shape)
    for idx in shape_order:
        XYs = paths_XYs[idx]
        for XY in XYs:
            for x, y in XY:
                if 0 <= int(x) < img_shape[1] and 0 <= int(y) < img_shape[0]:
                    blank_image[int(y), int(x)] = 0
    return blank_image

def detect_geometric_shapes(image):
    """Detect geometric shapes in the image."""
    shapes = {'rectangles': [], 'circles': []}
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    edges = cv2.Canny(gray, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if len(contour) >= 4:
            approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
            if len(approx) == 4:
                rect = cv2.minAreaRect(contour)
                box = np.int0(cv2.boxPoints(rect))
                shapes['rectangles'].append(box)
            elif len(approx) > 4:
                shapes['circles'].append(cv2.fitEllipse(contour))
    return shapes

def detect_symmetry(path_XYs):
    """Detect reflection symmetry in the given points."""
    def is_symmetric(points, axis):
        centroid = np.mean(points, axis=0)
        for point in points:
            reflected_point = point.copy()
            reflected_point[axis] = 2 * centroid[axis] - point[axis]
            if not np.any(np.all(np.isclose(points, reflected_point, atol=1e-6), axis=1)):
                return False
        return True

    centroid = np.mean(path_XYs, axis=0)
    if is_symmetric(path_XYs, axis=0): return True, 'x-axis'
    if is_symmetric(path_XYs, axis=1): return True, 'y-axis'
    return False, None

def detect_radial_symmetry(path_XYs):
    """Detect radial symmetry in the given points."""
    def is_radially_symmetric(points):
        centroid = np.mean(points, axis=0)
        angles = np.arctan2(points[:, 1] - centroid[1], points[:, 0] - centroid[0])
        angles = np.round(np.degrees(angles) % 360)
        unique_angles = np.unique(angles)
        if len(unique_angles) < 10:
            return True
        return False

    return is_radially_symmetric(path_XYs)

def complete_curve(path_XYs, degree=3, num_points=100):
    """Fit and visualize a polynomial curve to the given points."""
    if len(path_XYs) < 2: raise ValueError("Not enough points to fit a curve.")
    X, Y = path_XYs[:, 0], path_XYs[:, 1]
    polynomial_features = PolynomialFeatures(degree=degree)
    X_poly = polynomial_features.fit_transform(X.reshape(-1, 1))
    model = LinearRegression().fit(X_poly, Y)
    X_new = np.linspace(X.min(), X.max(), num_points)
    X_new_poly = polynomial_features.transform(X_new.reshape(-1, 1))
    Y_new = model.predict(X_new_poly)
    plt.scatter(X, Y, color='red', label='Original Points')
    plt.plot(X_new, Y_new, color='blue', label='Completed Curve')
    plt.legend()
    plt.title('Curve Fitting')
    plt.show()

def polylines2svg(paths_XYs, filename):
    """Convert polylines to SVG format and save."""
    dwg = svgwrite.Drawing(filename, profile='tiny')
    colours = ['black', 'red', 'green', 'blue', 'orange', 'purple', 'brown', 'cyan', 'magenta', 'yellow']
    for i, XYs in enumerate(paths_XYs):
        color = colours[i % len(colours)]
        for XY in XYs:
            points = [(x, y) for x, y in XY]
            dwg.add(dwg.polygon(points, fill=color, stroke='none'))
    dwg.save()

def save_to_csv(paths_XYs, filename):
    """Save polylines to CSV format."""
    with open(filename, 'w') as f:
        f.write('PolylineID,X,Y\n')
        for polyline_id, XYs in enumerate(paths_XYs):
            for XY in XYs:
                for x, y in XY:
                    f.write(f'{polyline_id},{x},{y}\n')

def main():
    # Adjust input and output file paths as necessary
    csv_path = "examples/occlusion2.csv"  # Update with the correct path if necessary
    base_filename = os.path.splitext(os.path.basename(csv_path))[0]
    output_csv_path = os.path.join("examples", f"{base_filename}_sol.csv")
    output_svg_path = os.path.join("examples", f"{base_filename}_sol.svg")
    output_image_path = os.path.join("examples", f"{base_filename}_composite.png")

    paths_XYs = read_csv(csv_path)

    # Determine image shape
    img_shape = (int(np.max([XY[:, 1].max() for XYs in paths_XYs for XY in XYs])) + 1,
                 int(np.max([XY[:, 0].max() for XYs in paths_XYs for XY in XYs])) + 1)

    # Create composite image
    shape_order = range(len(paths_XYs))
    composite_img = composite_shapes(paths_XYs, img_shape, shape_order)
    cv2.imwrite(output_image_path, composite_img)

    # Detect geometric shapes and print results
    shapes = detect_geometric_shapes(composite_img)
    print("Detected shapes:")
    for shape_type, detected in shapes.items():
        print(f"{shape_type.capitalize()}: {len(detected)} detected")

    # Analyze symmetry for each path
    for i, path_XYs in enumerate(paths_XYs):
        combined_XYs = np.vstack(path_XYs)
        reflection_symmetric, axis = detect_symmetry(combined_XYs)
        radial_symmetric = detect_radial_symmetry(combined_XYs)
        print(f"Path {i}: Reflection Symmetric along {axis if reflection_symmetric else 'None'}")
        print(f"Path {i}: Radially Symmetric {'Yes' if radial_symmetric else 'No'}")
        complete_curve(combined_XYs)

    # Save polylines to SVG and CSV
    polylines2svg(paths_XYs, output_svg_path)
    save_to_csv(paths_XYs, output_csv_path)

if __name__ == "__main__":
    main()
